In [1]:
from collections import OrderedDict

import numpy as np
from docplex.mp.model import Model
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance
from qiskit.aqua import aqua_globals
from qiskit.aqua.algorithms import VQE, NumPyMinimumEigensolver, QAOA
from qiskit.aqua.components.optimizers import SPSA
from qiskit.circuit.library import RealAmplitudes
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer

In [2]:
def get_time_matrix(machines_times, tasks):
    r = []
    for i in machines_times:
        tmp = []
        for j in tasks:
            tmp.append(j / i)
        r.append(tmp)
    return np.array(r)


def get_cost_matrix(tasks_execution_times, machines_costs):
    m = []
    for i in range(len(tasks_execution_times)):
        tmp = []
        for j in tasks_execution_times[i]:
            tmp.append(machines_costs[i] * j)
        m.append(tmp)
    return m

With 14 or more variables in the model, kernel is breaking down.

In this specific situation there are two qubits for each task -> 2*4=8 qubits needed.

First path minimum time is 1+2+4=7 log2(11-7)=log2(4) -> 3 qubits needed.

Second path minimum time is 1+3+4=8 log2(11-8)=log2(3) -> 2 qubits needed.

8+3+2=13 variables and with that number of variables, kernel works (slowly but works).

   0
  / \
 /   \
 1    2
 \    /
  \  /
    3

In [3]:
machines_times = [3, 6, 2]
machines_costs = [2, 5, 1]
tasks = [6, 12, 18, 24]
paths = [[0, 1, 3], [0, 2, 3]]
d = 11

time_matrix = np.array(get_time_matrix(machines_times, tasks))
cost_matrix = np.array(get_cost_matrix(time_matrix, machines_costs))

print("Time matrix:\n {}".format(time_matrix))
print("Cost matrix:\n {}".format(cost_matrix))

Time matrix:
 [[ 2.  4.  6.  8.]
 [ 1.  2.  3.  4.]
 [ 3.  6.  9. 12.]]
Cost matrix:
 [[ 4.  8. 12. 16.]
 [ 5. 10. 15. 20.]
 [ 3.  6.  9. 12.]]


In [4]:
# Correct only for three machines!
correct_machines = ['00', '01', '11']
machine_to_index = {'00': 0, '01': 1, '11': 2}

def get_task_subvector(vector, task_index):
    x = len(machines_costs) - 1  # E.g., when 3 machines, then subvector for each task has length of 2 (domain wall rule).
    subvector = ''
    for i in range(2 * task_index, 2 * task_index + x):
        bit_value = str(int(vector[i]))
        subvector += bit_value
    return subvector


def solution_vector_correct(vector):
    for task in range(len(tasks)):
        if get_task_subvector(vector, task) not in correct_machines:
            return False
    return True


def execution_times_not_bigger_than_deadline(vector):
    for path in paths:
        path_time_sum = 0
        for task in path:
            task_machine = machine_to_index.get(get_task_subvector(vector, task))
            path_time_sum += time_matrix[task_machine, task]
        if path_time_sum > d:
            return False

    return True


def is_solution_correct(vector):
    return solution_vector_correct(vector) and execution_times_not_bigger_than_deadline(vector)

Slack variables are no longer needed. Qiskit is creating them for us. 

Still we have to remember about the problem not being too big. With >=14 variables, kernel will break down.

However maybe there is some workaround? 

In [5]:
def get_quadratic_problem():
    a = len(tasks)  # Number of tasks.
    b = len(machines_costs) - 1  # Size of vector needed to encode the machine number.
    
    mdl = Model(name='workflow')
    x = {(i, j): mdl.binary_var(name='x_{0}_{1}'.format(i, j)) for i in range(0, a) for j in range(0, b)}
    
    objective = mdl.sum(cost_matrix[2, i] * x[(i, 0)]
                        + cost_matrix[1, i] * (x[(i, 1)] - x[(i, 0)]) ** 2
                        + cost_matrix[0, i] * (1 - x[(i, 1)]) for i in range(0, a))

    mdl.minimize(objective)

    for k in range(0, len(paths)):
        mdl.add_constraint(mdl.sum([time_matrix[2, i] * x[(i, 0)]
                                + time_matrix[1, i] * (x[(i, 1)] - x[(i, 0)])
                                + time_matrix[0, i] * (1 - x[(i, 1)])
                                for i in paths[k]]) <= d, "deadline_path_{}".format(k))

    qp = QuadraticProgram()
    qp.from_docplex(mdl)
    return qp

In [6]:
operator = get_quadratic_problem()

In [7]:
aqua_globals.random_seed = 10598
# Creating two instances just to make sure they are not somehow 'bounded', idk if that makes sense.
quantum_instance_vqe = QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                       seed_simulator=aqua_globals.random_seed,
                                       seed_transpiler=aqua_globals.random_seed)

quantum_instance_qaoa = QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                        seed_simulator=aqua_globals.random_seed,
                                        seed_transpiler=aqua_globals.random_seed)

exact_mes = NumPyMinimumEigensolver()
r_a = RealAmplitudes(operator.get_num_binary_vars(), reps=2, entanglement='full')
vqe_mes = VQE(quantum_instance=quantum_instance_vqe, var_form=r_a, optimizer=SPSA(maxiter=1000))
qaoa_mes = QAOA(quantum_instance=quantum_instance_qaoa, initial_point=[0., 0.])

exact = MinimumEigenOptimizer(exact_mes)  
vqe = MinimumEigenOptimizer(vqe_mes)
qaoa = MinimumEigenOptimizer(qaoa_mes) 
recursive = RecursiveMinimumEigenOptimizer(min_eigen_optimizer=qaoa, min_num_vars=6, min_num_vars_optimizer=qaoa)

In [8]:
def perform_optimization(optimization, qubitOp, name):
    result = optimization.solve(qubitOp)
    print_optimization_output(result, name)

In [9]:
def print_optimization_output(result, solver_name):
    print("------ %s ------" % solver_name)
    print(result)
    print("Is the found vector correct?: {}".format(is_solution_correct(result.x)))
    print("Most probable sample: {}".format(max(result.samples, key=lambda item: item[2])))
    print("Samples count: {}".format(len(result.samples)))

In [10]:
perform_optimization(exact, operator, "Numpy Eigensolver")

------ Numpy Eigensolver ------
optimal function value: 43.0
optimal value: [0. 1. 1. 1. 0. 0. 0. 1.]
status: SUCCESS
Is the found vector correct?: True
Most probable sample: ('0111000100000', 43.0, 1.0)
Samples count: 1


In [11]:
perform_optimization(vqe, operator, "VQE")

------ VQE ------
optimal function value: 43.0
optimal value: [0. 1. 1. 1. 0. 0. 0. 1.]
status: SUCCESS
Is the found vector correct?: True
Most probable sample: ('0001010100100', 2217.0, 0.17378811282069348)
Samples count: 867


In [12]:
perform_optimization(qaoa, operator, "QAOA")

------ QAOA ------
optimal function value: 43.0
optimal value: [0. 1. 1. 1. 0. 0. 0. 1.]
status: SUCCESS
Is the found vector correct?: True
Most probable sample: ('1010010100000', 6846.0, 0.0001220703125000005)
Samples count: 8192


In [ ]:
perform_optimization(recursive, operator, "Recursive")